In [1]:
import pandas as pd
import numpy as np
import os
import glob
import json
from joblib import dump, load
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor

## Configuration

In [2]:
# Number of force cells in the robotic leg
N_CELLS = 8

# Path where the results are stored
RESULTS_PATH = '../../../results'
# ID of the training and test data resulting from this notebook, stored in RESULTS_PATH
DATA_ID = '0003_11042021'
# Hyperparameters search date
HS_DATE = '12042021'
# Number of folds in cross-validation
CV = 6

print('Model trained with data: ' + DATA_ID)

pd.set_option('display.max_columns', None)

Model trained with data: 0003_11042021


## Hyperparameters seach analysis

In [3]:
results_files_ls = glob.glob(os.path.join(RESULTS_PATH, DATA_ID, 'RF_{}'.format(HS_DATE), 'RF_{}_*.json'.format(HS_DATE)))

print('Number of results files: {}'.format(len(results_files_ls)))

Number of results files: 189


In [4]:
# Load all the results and generates a pandas dataframe
results_ls = []
for results_file in results_files_ls:
    with open(results_file) as json_file:
        results_dict = json.load(json_file)
        
    dict_aux = {}
    dict_aux['params_ID'] = results_dict['id']
    for key, value in results_dict['parameters'].items():
        dict_aux['param_' + key] = value
    for key, value in results_dict['cv_results'].items():
        dict_aux['__'.join([key, 'mean'])] = np.mean(value)
        dict_aux['__'.join([key, 'std'])] = np.std(value)

    results_ls.append(dict_aux)
        
results_df = pd.DataFrame(results_ls)
results_df

,params_ID,param_max_depth,param_max_features,param_min_samples_leaf,param_min_samples_split,param_n_estimators,fit_time__mean,fit_time__std,Train_Fx_MAE_mean__mean,Train_Fx_MAE_mean__std,Train_Fx_MAE_std__mean,Train_Fx_MAE_std__std,Train_Fx_MSE_mean__mean,Train_Fx_MSE_mean__std,Train_Fx_MSE_std__mean,Train_Fx_MSE_std__std,Train_Fx_R2_mean__mean,Train_Fx_R2_mean__std,Train_Fx_R2_std__mean,Train_Fx_R2_std__std,Train_Fy_MAE_mean__mean,Train_Fy_MAE_mean__std,Train_Fy_MAE_std__mean,Train_Fy_MAE_std__std,Train_Fy_MSE_mean__mean,Train_Fy_MSE_mean__std,Train_Fy_MSE_std__mean,Train_Fy_MSE_std__std,Train_Fy_R2_mean__mean,Train_Fy_R2_mean__std,Train_Fy_R2_std__mean,Train_Fy_R2_std__std,Train_Fz_MAE_mean__mean,Train_Fz_MAE_mean__std,Train_Fz_MAE_std__mean,Train_Fz_MAE_std__std,Train_Fz_MSE_mean__mean,Train_Fz_MSE_mean__std,Train_Fz_MSE_std__mean,Train_Fz_MSE_std__std,Train_Fz_R2_mean__mean,Train_Fz_R2_mean__std,Train_Fz_R2_std__mean,Train_Fz_R2_std__std,Test_Fx_MAE_mean__mean,Test_Fx_MAE_mean__std,Test_Fx_MAE_std__mean,Test_Fx_MAE_std__std,Test_Fx_MSE_mean__mean,Test_Fx_MSE_mean__std,Test_Fx_MSE_std__mean,Test_Fx_MSE_std__std,Test_Fx_R2_mean__mean,Test_Fx_R2_mean__std,Test_Fx_R2_std__mean,Test_Fx_R2_std__std,Test_Fy_MAE_mean__mean,Test_Fy_MAE_mean__std,Test_Fy_MAE_std__mean,Test_Fy_MAE_std__std,Test_Fy_MSE_mean__mean,Test_Fy_MSE_mean__std,Test_Fy_MSE_std__mean,Test_Fy_MSE_std__std,Test_Fy_R2_mean__mean,Test_Fy_R2_mean__std,Test_Fy_R2_std__mean,Test_Fy_R2_std__std,Test_Fz_MAE_mean__mean,Test_Fz_MAE_mean__std,Test_Fz_MAE_std__mean,Test_Fz_MAE_std__std,Test_Fz_MSE_mean__mean,Test_Fz_MSE_mean__std,Test_Fz_MSE_std__mean,Test_Fz_MSE_std__std,Test_Fz_R2_mean__mean,Test_Fz_R2_mean__std,Test_Fz_R2_std__mean,Test_Fz_R2_std__std
0,36LZKCUXPX,13,0.2,0.010,0.0100,1000,38.531896,0.292728,8.293597,0.269200,2.320279,0.092058,139.426954,8.553446,70.675360,3.545023,0.597713,0.021375,0.060194,0.010535,8.491057,0.206025,5.914225,0.253472,223.625125,11.640757,250.455256,17.344791,0.468452,0.015957,0.111703,0.009999,10.782770,0.405617,3.841568,0.246229,272.251313,21.912704,220.031718,31.247252,0.581321,0.022405,0.068758,0.014736,12.248514,1.468944,4.288143,0.729241,299.741081,73.875420,199.434367,57.616807,-0.002072,0.295405,0.252135,0.060772,11.281859,1.817362,8.714458,1.739269,429.737166,136.841507,641.317243,180.387693,-0.063488,0.291136,0.413969,0.493538,16.220895,2.878738,6.707404,1.552511,599.830546,225.920637,574.990043,338.018971,-0.280532,0.311635,0.725191,0.439405
1,WF84XYZ7X6,15,0.2,0.001,0.0001,100,5.584132,0.055184,3.989016,0.216867,1.032845,0.068532,40.075266,4.704387,17.793164,2.011311,0.881916,0.012370,0.027325,0.005003,4.386793,0.090987,2.961924,0.114399,63.063597,2.735547,67.510873,4.402733,0.824512,0.010420,0.053039,0.004350,4.974872,0.216206,1.797059,0.134057,66.075079,6.296866,42.509821,6.346733,0.891425,0.008484,0.029403,0.003634,11.820579,1.762237,4.023486,0.974601,292.937572,90.624492,189.939796,80.309630,0.004759,0.339559,0.280261,0.113298,11.540467,1.832232,8.883276,1.977760,452.815338,149.954984,642.917186,188.426411,-0.212758,0.434563,0.660147,0.908696,15.699374,3.063488,6.706369,1.735746,590.315274,229.493714,568.157855,305.878718,-0.280355,0.409758,0.802198,0.633418
2,SDUEIILGAK,13,0.3,0.100,0.0100,5000,90.850062,3.026931,11.857662,0.221253,3.881044,0.143315,276.128594,8.270421,170.340212,5.352094,0.233624,0.028285,0.073646,0.012372,10.874762,0.290503,7.655420,0.351103,393.822840,18.403318,483.010909,32.685227,0.158749,0.022415,0.070464,0.012522,15.648575,0.519284,5.486778,0.333128,545.148437,43.343965,479.314836,77.110240,0.177893,0.027113,0.092930,0.010166,13.451048,1.117708,5.499440,0.852936,349.730815,56.666686,263.210643,63.415569,-0.102518,0.123107,0.285219,0.123479,11.502202,1.941588,8.898177,2.165522,452.338312,151.367316,728.910645,249.040555,-0.017960,0.132479,0.165428,0.097025,17.696312,2.791643,6.878257,1.283146,665.362901,263.778701,621.664918,439.415503,-0.436281,0.203747,0.780843,0.314696
3,OKC90NRKB5,7,0.3,0.100,0

In [5]:
# Sum up the scores by force axis in only one sortable score
for subset in ['Train', 'Test']:
    for loss in ['MAE', 'MSE', 'R2']:
        results_df[subset + '_' + loss] = results_df[[subset + '_' + force + '_' + loss + '_mean__mean' for force in ['Fx', 'Fy', 'Fz']]].mean(axis=1)

In [6]:
# Sort the dataframe by the most relevant score
results_df = results_df.sort_values(['Test_R2'], ascending=False)
results_df

,params_ID,param_max_depth,param_max_features,param_min_samples_leaf,param_min_samples_split,param_n_estimators,fit_time__mean,fit_time__std,Train_Fx_MAE_mean__mean,Train_Fx_MAE_mean__std,Train_Fx_MAE_std__mean,Train_Fx_MAE_std__std,Train_Fx_MSE_mean__mean,Train_Fx_MSE_mean__std,Train_Fx_MSE_std__mean,Train_Fx_MSE_std__std,Train_Fx_R2_mean__mean,Train_Fx_R2_mean__std,Train_Fx_R2_std__mean,Train_Fx_R2_std__std,Train_Fy_MAE_mean__mean,Train_Fy_MAE_mean__std,Train_Fy_MAE_std__mean,Train_Fy_MAE_std__std,Train_Fy_MSE_mean__mean,Train_Fy_MSE_mean__std,Train_Fy_MSE_std__mean,Train_Fy_MSE_std__std,Train_Fy_R2_mean__mean,Train_Fy_R2_mean__std,Train_Fy_R2_std__mean,Train_Fy_R2_std__std,Train_Fz_MAE_mean__mean,Train_Fz_MAE_mean__std,Train_Fz_MAE_std__mean,Train_Fz_MAE_std__std,Train_Fz_MSE_mean__mean,Train_Fz_MSE_mean__std,Train_Fz_MSE_std__mean,Train_Fz_MSE_std__std,Train_Fz_R2_mean__mean,Train_Fz_R2_mean__std,Train_Fz_R2_std__mean,Train_Fz_R2_std__std,Test_Fx_MAE_mean__mean,Test_Fx_MAE_mean__std,Test_Fx_MAE_std__mean,Test_Fx_MAE_std__std,Test_Fx_MSE_mean__mean,Test_Fx_MSE_mean__std,Test_Fx_MSE_std__mean,Test_Fx_MSE_std__std,Test_Fx_R2_mean__mean,Test_Fx_R2_mean__std,Test_Fx_R2_std__mean,Test_Fx_R2_std__std,Test_Fy_MAE_mean__mean,Test_Fy_MAE_mean__std,Test_Fy_MAE_std__mean,Test_Fy_MAE_std__std,Test_Fy_MSE_mean__mean,Test_Fy_MSE_mean__std,Test_Fy_MSE_std__mean,Test_Fy_MSE_std__std,Test_Fy_R2_mean__mean,Test_Fy_R2_mean__std,Test_Fy_R2_std__mean,Test_Fy_R2_std__std,Test_Fz_MAE_mean__mean,Test_Fz_MAE_mean__std,Test_Fz_MAE_std__mean,Test_Fz_MAE_std__std,Test_Fz_MSE_mean__mean,Test_Fz_MSE_mean__std,Test_Fz_MSE_std__mean,Test_Fz_MSE_std__std,Test_Fz_R2_mean__mean,Test_Fz_R2_mean__std,Test_Fz_R2_std__mean,Test_Fz_R2_std__std,Train_MAE,Train_MSE,Train_R2,Test_MAE,Test_MSE,Test_R2
159,1KGYR9IE8V,15,0.2,0.010,0.0100,100,3.907211,0.041695,8.250258,0.286284,2.293878,0.101546,138.328332,8.990843,69.711930,3.716997,0.600403,0.021955,0.060052,0.009706,8.462369,0.190287,5.884515,0.241281,222.166744,10.742809,248.041411,15.629646,0.470333,0.017016,0.112322,0.008769,10.739287,0.399360,3.824509,0.250649,271.001447,21.902364,218.370869,31.965609,0.583117,0.022469,0.069386,0.013905,12.212270,1.504909,4.245837,0.704140,298.890833,75.587088,197.595075,57.460764,-0.001022,0.307129,0.248707,0.064487,11.251960,1.819990,8.660540,1.690955,423.984063,136.331501,629.658575,177.589440,-0.054230,0.286978,0.401921,0.460208,16.232581,2.895135,6.712613,1.565798,599.850009,225.292725,575.037872,334.109376,-0.279363,0.317695,0.720009,0.428534,9.150638,210.498841,0.551284,13.232270,440.908302,-0.111538
109,T7S2Z8YC4Y,15,0.2,0.010,0.0010,100,3.974562,0.077967,8.250258,0.286284,2.293878,0.101546,138.328332,8.990843,69.711930,3.716997,0.600403,0.021955,0.060052,0.009706,8.462369,0.190287,5.884515,0.241281,222.166744,10.742809,248.041411,15.629646,0.470333,0.017016,0.112322,0.008769,10.739287,0.399360,3.824509,0.250649,271.001447,21.902364,218.370869,31.965609,0.583117,0.022469,0.069386,0.013905,12.212270,1.504909,4.245837,0.704140,298.890833,75.587088,197.595075,57.460764,-0.001022,0.307129,0.248707,0.064487,11.251960,1.819990,8.660540,1.690955,423.984063,136.331501,629.658575,177.589440,-0.054230,0.286978,0.401921,0.460208,16.232581,2.895135,6.712613,1.565798,599.850009,225.292725,575.037872,334.109376,-0.279363,0.317695,0.720009,0.428534,9.150638,210.498841,0.551284,13.232270,440.908302,-0.111538
25,04LYEPEJY2,13,0.2,0.010,0.0001,100,4.002557,0.091523,8.251228,0.285054,2.294159,0.100303,138.361733,8.950949,69.729092,3.648570,0.600288,0.021983,0.060050,0.009765,8.464398,0.189494,5.885984,0.240626,222.247240,10.734424,248.126032,15.611168,0.470232,0.016849,0.112188,0.008699,10.741285,0.397947,3.824706,0.250248,271.018307,21.723134,218.306379,31.719268,0.582983,0.022402,0.069468,0.013964,12.212815,1.513007,4.247029,0.707139,299.001789,75.823487,197.816911,57.605714,-0.001295,0.307930,0.249053,0.064459,11.253036,1.819916,8.660282,1.687628,424.057786,136.557716,629.576392,177.9145

In [7]:
best_params = dict(results_df.iloc[0][[col for col in results_df.columns if 'param_' in col]])
best_params = {key.replace('param_', ''): value for key, value in best_params.items()}
print('Best parameters: {}'.format(best_params))

Best parameters: {'max_depth': 15, 'max_features': 0.2, 'min_samples_leaf': 0.01, 'min_samples_split': 0.01, 'n_estimators': 100}


## Best model

In [8]:
# Load data
X_train = np.load(os.path.join(RESULTS_PATH, DATA_ID, 'data', 'X_train_{}.npy'.format(DATA_ID)))
X_test = np.load(os.path.join(RESULTS_PATH, DATA_ID, 'data', 'X_test_{}.npy'.format(DATA_ID)))
Y_train = np.load(os.path.join(RESULTS_PATH, DATA_ID, 'data', 'Y_train_{}.npy'.format(DATA_ID)))
Y_test = np.load(os.path.join(RESULTS_PATH, DATA_ID, 'data', 'Y_test_{}.npy'.format(DATA_ID)))

In [10]:
# Setup the model with the best parameters
model = RandomForestRegressor(**best_params, random_state=0, n_jobs=-1, verbose=1)

model.fit(X_train, Y_train)

# Save the model
dump(model, os.path.join(RESULTS_PATH, DATA_ID, 'RF_{}'.format(HS_DATE), 'RF_best_model_{}_{}.joblib'.format(HS_DATE, DATA_ID))) 

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    5.0s finished


['../../../results/0003_11042021/RF_12042021/RF_best_model_12042021_0003_11042021.joblib']

In [11]:
train_preds = model.predict(X_train)
test_preds = model.predict(X_test)

results = {
    'Train': {
        'MAE': mean_absolute_error(Y_train, train_preds, multioutput='raw_values'),
        'MSE': mean_squared_error(Y_train, train_preds, multioutput='raw_values'),
        'R2': r2_score(Y_train, train_preds, multioutput='raw_values')
    },
    'Test': {
        'MAE': mean_absolute_error(Y_test, test_preds, multioutput='raw_values'),
        'MSE': mean_squared_error(Y_test, test_preds, multioutput='raw_values'),
        'R2': r2_score(Y_test, test_preds, multioutput='raw_values')
    }       
    
}

# # Display the score for each axis of each force cell
# for subset in ['Train', 'Test']:
#     for f, force in enumerate(['Fx', 'Fy', 'Fz']):
#         for c in range(N_CELLS):
#             for loss in ['MAE', 'MSE', 'R2']:
#                 scores = [results[subset][loss][i + f] for i in range(0, N_CELLS * 3, 3)]
#                 print('{} {}{}{} {}: {:.4f}'.format(subset, force[0], c + 1, force[-1], loss, scores[c]))
# print('\n')

# Display the score mean and standard deviation of each axis
for subset in ['Train', 'Test']:
    for f, force in enumerate(['Fx', 'Fy', 'Fz']):
        for loss in ['MAE', 'MSE', 'R2']:
            scores = [results[subset][loss][i + f] for i in range(0, N_CELLS * 3, 3)]
            print(' '.join([subset, force, loss]) + ': {:.4f} ± {:.4f}'.format(np.mean(scores), np.std(scores)))

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.5s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.1s finished


Train Fx MAE: 8.6461 ± 2.4048
Train Fx MSE: 151.6214 ± 77.8937
Train Fx R2: 0.5655 ± 0.0612
Train Fy MAE: 8.7525 ± 6.0748
Train Fy MSE: 238.7282 ± 265.7849
Train Fy R2: 0.4389 ± 0.1117
Train Fz MAE: 11.2891 ± 3.9985
Train Fz MSE: 300.2048 ± 246.7115
Train Fz R2: 0.5458 ± 0.0617
Test Fx MAE: 13.0032 ± 5.0952
Test Fx MSE: 311.5999 ± 249.7830
Test Fx R2: 0.3473 ± 0.2863
Test Fy MAE: 10.7406 ± 7.7562
Test Fy MSE: 378.7206 ± 481.6318
Test Fy R2: 0.3404 ± 0.1027
Test Fz MAE: 19.0024 ± 6.5255
Test Fz MSE: 687.2684 ± 593.5296
Test Fz R2: 0.3611 ± 0.2479


In [ ]:
# plt.figure(figsize=(20,15))
# plt.scatter(Y_train[:, 3], Y_train[:, 4], label='true', alpha=0.3)
# plt.scatter(train_preds[:, 3], train_preds[:, 4], label='preds', alpha=0.3)
# plt.legend()
# plt.show()

# plt.figure(figsize=(20,15))
# plt.scatter(Y_train[:100, 3], Y_train[:100, 4], label='true', alpha=0.3)
# plt.scatter(train_preds[:100, 3], train_preds[:100, 4], label='preds', alpha=0.3)
# plt.legend()
# plt.show()

# plt.figure(figsize=(20,15))
# plt.scatter(Y_test[:, 3], Y_test[:, 4], label='true', alpha=0.3)
# plt.scatter(test_preds[:, 3], test_preds[:, 4], label='preds', alpha=0.3)
# plt.legend()
# plt.show()